In [ ]:
import importlib 
import textoABraille as tb
import pipeline as pl
from PIL import Image

import pytesseract

pytesseract.pytesseract.tesseract_cmd = r'D:\Tesseract\tesseract.exe' 

In [ ]:
importlib.reload(tb) #Para actualizar los cambios en el archivo textoABraille.py
importlib.reload(pl) #Para actualizar los cambios en el archivo pipeline.py

In [ ]:
ejemplo="Hola, Pepino"
print(tb.convertirTextoABraille(ejemplo))

In [ ]:
# Prueba de texto a señales
braille_signals = tb.convertirASeñales("Hola, Pepino")
binary_signals = [format(signal, '06b') for signal in braille_signals]
print(binary_signals)

In [ ]:
#Prueba de imagen (captura de pantalla) a braille
image = Image.open('../ImagenesPrueba/Screenshot_3.png')
braille = pl.convertirImagen(image, debug=True)
print(braille)

## Cámara

In [ ]:
import cv2
import pytesseract
from pytesseract import Output
# Open the default camera
# cam = cv2.VideoCapture(0)
cam = cv2.VideoCapture("http://10.101.48.143:81/stream")

# Get the default frame width and height
frame_width = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
# out = cv2.VideoWriter('output.mp4', fourcc, 20.0, (frame_width, frame_height))

counter = 0
ocr_interval = 60
resize_factor = 4
boxes_and_text = []
while True:
    # if counter == 5:
    #     break
    ret, frame = cam.read()

    if not ret:
        print("No se pudo capturar el frame")
        break

    
    if counter % ocr_interval == 0:
        processed_frame = pl.preprocesarImagen(frame, resize_factor=resize_factor)
        d = pytesseract.image_to_data(processed_frame, output_type=Output.DICT, lang='spa')
        boxes_and_text.clear()
        # Cajas de texto detectadas 
        n_boxes = len(d['level'])
        for i in range(n_boxes):
            (x, y, w, h) = (d['left'][i] / resize_factor, d['top'][i] / resize_factor, d['width'][i] / resize_factor, d['height'][i] / resize_factor)
            x, y, w, h = int(x), int(y), int(w), int(h)
            text = d['text'][i]
            text = text.strip().replace('\n', ' ')
            if len(text) < 3:
                continue
            if text:
                # print(f"Detected text: {text}")
                boxes_and_text.append((x, y, w, h, text))
                # cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                # cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
                braille = tb.convertirTextoABraille(text)q
                print(f"Se detectó el texto: {text} y se convirtió a braille: {braille}")
                # cv2.putText(frame, braille, (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
    cv2.imshow('Processed', processed_frame)

    for (x, y, w, h, text) in boxes_and_text:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)



    # Display the captured frame
    cv2.imshow('Camera', frame)
    counter += 1
    # Press 'q' to exit the loop
    if cv2.waitKey(1) == ord('q'):
        break
    elif cv2.waitKey(1) == ord('s'):
        cv2.imwrite(f'frame_{counter}.png', frame)
        print(f"Frame saved as frame_{counter}.png")

# Release the capture and writer objects
cam.release()
# out.release()
cv2.destroyAllWindows()

## Intento 2

In [32]:
import cv2
import pytesseract
from pytesseract import Output
from PIL import Image
import numpy as np
import textoABraille as tb
import serial
import textoABraille as tb
import pipeline as pl

importlib.reload(tb) #Para actualizar los cambios en el archivo textoABraille.py
importlib.reload(pl) #Para actualizar los cambios en el archivo pipeline.py
# Set the path to the Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'D:\Tesseract\tesseract.exe'  # Change to the path of Tesseract on your computer or comment if already in PATH

# Initialize serial communication with ESP32
# ser = serial.Serial('COM3', 115200)  # Change 'COM3' to the appropriate port


def send_signals_to_esp32(braille_text):
    # Convert Braille text to servo motor angles
    angles = [ord(char) for char in braille_text]  # Example conversion
    for angle in angles:
        ser.write(f'{angle}\n'.encode())
        print(f'Sent angle: {angle}')

def main(file_path):
    if file_path.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
        text = pl.process_image(file_path)
    elif file_path.lower().endswith('.pdf'):
        text = pl.process_pdf(file_path)
    else:
        print("Unsupported file format")
        return
    # print(f"Detected text: {text}")
    braille_text = tb.convertirAAngulos(text)
    print(f"Detected text: {text}")
    print(f"Converted to Braille: {braille_text}")
    # send_signals_to_esp32(braille_text)

main('..\ImagenesPrueba/imagen.pdf')

Detected text: El gato ya no se encuentra en la casa.

Converted to Braille: [[90, 45], [157.5, 0], [0, 0], [135, 135], [90, 0], [67.5, 135], [112.5, 45], [0, 0], [112.5, 157.5], [90, 0], [0, 0], [112.5, 135], [112.5, 45], [0, 0], [67.5, 90], [90, 45], [0, 0], [90, 45], [112.5, 135], [90, 90], [112.5, 22.5], [90, 45], [112.5, 135], [67.5, 135], [157.5, 45], [90, 0], [0, 0], [90, 45], [112.5, 135], [0, 0], [157.5, 0], [90, 0], [0, 0], [90, 90], [90, 0], [67.5, 90], [90, 0], [22.5, 0]]
